In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,82.38,81,40,11.50,US,1659244472
1,1,busselton,-33.6500,115.3333,57.16,69,71,11.86,AU,1659244486
2,2,bengkulu,-3.8004,102.2655,84.67,64,99,5.39,ID,1659244188
3,3,along,28.1667,94.7667,77.67,96,98,2.44,IN,1659244913
4,4,ponta do sol,32.6667,-17.1000,70.72,82,12,5.26,PT,1659244913


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,manicore,-5.8092,-61.3003,71.55,94,0,1.14,BR,1659244931
136,136,westport,41.1415,-73.3579,72.75,80,0,1.99,US,1659244539
152,152,albertville,34.2676,-86.2089,74.07,92,0,3.44,US,1659244965
165,165,lagoa,39.0500,-27.9833,72.03,72,0,4.45,PT,1659244969
190,190,iberia,29.9502,-91.7507,78.87,88,0,5.75,US,1659244977
202,202,oakdale,44.9630,-92.9649,74.62,74,0,8.05,US,1659244968
209,209,nijar,36.9665,-2.2060,75.65,89,0,3.44,ES,1659244982
224,224,warwick,41.7001,-71.4162,72.90,74,0,4.61,US,1659244889
237,237,bethel,41.3712,-73.4140,71.64,83,0,0.00,US,1659244483
242,242,bastia,42.6667,9.3333,79.30,52,0,4.61,FR,1659244992


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
59,manicore,BR,-5.8092,-61.3003,
136,westport,US,41.1415,-73.3579,
152,albertville,US,34.2676,-86.2089,
165,lagoa,PT,39.0500,-27.9833,
190,iberia,US,29.9502,-91.7507,
202,oakdale,US,44.9630,-92.9649,
209,nijar,ES,36.9665,-2.2060,
224,warwick,US,41.7001,-71.4162,
237,bethel,US,41.3712,-73.4140,
242,bastia,FR,42.6667,9.3333,


In [9]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
59,manicore,BR,-5.8092,-61.3003,Hotel Durval
136,westport,US,41.1415,-73.3579,The Westport Inn
152,albertville,US,34.2676,-86.2089,Microtel Inn & Suites by Wyndham Albertville
165,lagoa,PT,39.0500,-27.9833,Casa Das Faias
190,iberia,US,29.9502,-91.7507,back to back rv park
202,oakdale,US,44.9630,-92.9649,Hilton Garden Inn St. Paul/Oakdale
209,nijar,ES,36.9665,-2.2060,La Casa del Cine
224,warwick,US,41.7001,-71.4162,Hampton Inn & Suites Providence/Warwick-Airport
237,bethel,US,41.3712,-73.4140,Courtyard by Marriott Danbury
242,bastia,FR,42.6667,9.3333,Hotel Santa Maria


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))